In [8]:
import numpy as np
import pandas as pd
import lightkurve as lk
import matplotlib.pyplot as plt
import re
import os
from astropy.io import fits

import warnings

warnings.filterwarnings("ignore")

In [7]:
path_data = '../results/TPF_dataframe.csv'
df = pd.read_csv(path_data)

folder = '../results/TPF_data/'
# list dubfolders in folder
subfolders = [f.path for f in os.scandir(folder) if f.is_dir()]

In [52]:
for index, row in df.iterrows():
    # Check if TPF was found for this star
    if row['TPF_Found']:
        # Create a folder for the star
        name = row['Star_Name']
        
        #join name with underscores
        name = name.replace(" ", "_")
        star_folder = f"{folder}{name}"

        if star_folder in subfolders:
            print(f"Folder for {name} already exists. Skipping...")
            continue

        os.makedirs(star_folder, exist_ok=True)
        # Search for the TPF
        found_name = row['Found_Star_Name']
        tpf = lk.search_targetpixelfile(row['Found_Star_Name'])
        n_tpf = len(tpf)

        print(f'{name} ({found_name})', end=': ')
        print(f"Found {n_tpf} TPFs")
        # Loop through the search result
        for i in range(n_tpf):
            print(f"Downloading TPF {i+1}/{n_tpf}", end='\r')
            # Get the exposure time
            exptime = tpf[i].exptime
            match = re.search(r'\d+', str(exptime))
            number = int(match.group())
            # Create a folder for this exposure time within the star's folder
            exptime_folder = f"{star_folder}/exp_{number}"
            os.makedirs(exptime_folder, exist_ok=True)

            try:
                tpf_file = tpf[i].download()
                fits_hdu = tpf_file.to_lightcurve(aperture_mask=tpf_file.pipeline_mask).to_fits()
                header = fits_hdu[0].header
                telescope, date, object = header['TELESCOP'], header['DATE'], header['OBJECT']
                path = f"{exptime_folder}/{name}_{telescope}_{date}_{object}_{i}.fits"
                fits_hdu.writeto(path, overwrite=True)
                
            except lk.LightkurveError as e:
                print(f"Error downloading TPF for {row['Star_Name']}: {e}")
            except FileNotFoundError as e:
                print(f"Error downloading TPF for {row['Star_Name']}: {e}")
            except IndexError as e:
                print(f"IndexError: {e}. Skipping iteration {i}.")

Kepler-1663 (Gaia DR2 2129578826996503040): Found 50 TPFs
IndexError: list index out of range. Skipping iteration 25.
IndexError: list index out of range. Skipping iteration 34.
K2-33 (K2-33): Found 1 TPFs


TOI-1227 (TOI-1227): Found 11 TPFs
HD_114082 (HD 114082): Found 4 TPFs
HIP_67522 (HIP 67522): Found 6 TPFs
WASP-25 (WASP-25): Found 2 TPFs
AU_Mic (AU Mic): Found 5 TPFs
V1298_Tau (V1298 Tau): Found 7 TPFs
HD_109833 (HD 109833): Found 8 TPFs
CoRoT-18 (CoRoT-18): Found 5 TPFs
TOI-837 (TOI-837): Found 12 TPFs
KOI-7913_A (TIC 158320130): Found 10 TPFs
KOI-7368 (KOI-7368): Found 26 TPFs
DS_Tuc_A (HD 222259 A): Found 12 TPFs
Kepler-1643 (Kepler-1643): Found 17 TPFs
TOI-942 (TOI-942): Found 3 TPFs
CoRoT-20 (CoRoT-20): Found 5 TPFs
Kepler-1928 (Kepler-1928): Found 23 TPFs
Kepler-970 (Kepler-970): Found 17 TPFs
K2-284 (K2-284): Found 7 TPFs
TOI-251 (TOI-251): Found 6 TPFs
Kepler-529 (Kepler-529): Found 18 TPFs
KOI-1783 (KOI-1783): Found 22 TPFs
Kepler-1733 (TIC 122781795): Found 17 TPFs
Kepler-279 (Kepler-279): Found 35 TPFs
Kepler-1903 (TIC 417655814): Found 17 TPFs
Kepler-1764 (TIC 138431522): Found 17 TPFs
Kepler-394 (Kepler-394): Found 34 TPFs
Kepler-59 (Kepler-59): Found 31 TPFs
